## Patrickwalsh1995@gmail.com
### https://www.drivendata.org/users/PatW42/

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from sklearn.metrics import confusion_matrix
import catboost as cb
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
pd.set_option('display.max_columns', 38)
DATA_DIR = "C:/Users/YOUR_NAME/Desktop/Richter's Predictor"

## Data import

In [ ]:
train_values = pd.read_csv(DATA_DIR +'/train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR +'/train_labels.csv', index_col='building_id')

## Preview the data

In [ ]:
train_values.head(5)

## Train Dataset Profile

In [ ]:
df_profile=train_values.join(train_labels)
train_profile=df_profile.profile_report(title="Richter's Train Dataset Pandas profile")
train_profile.to_file(output_file=DATA_DIR+"/Richter's Train Dataset Pandas profile.html")

In [ ]:
train_profile

## Test Dataset Profile

In [ ]:
test_values = pd.read_csv(DATA_DIR+'/test_values.csv', index_col='building_id')
test_profile=test_values.profile_report(title="Richter's Test Dataset Pandas profile")
test_profile.to_file(output_file=DATA_DIR+"/Richter's Test Dataset Pandas profile.html")
test_profile

In [ ]:
(train_labels.damage_grade
             .value_counts()
             .sort_index()
             .plot.bar(title="Number of Buildings with Each Damage Grade"))

In [ ]:
#correlation heat map
sns.heatmap(train_values.join(train_labels).corr(), annot=False, fmt=".2f")
plt.show()

## Preprocessing

In [ ]:
cat_cols=['geo_level_1_id','geo_level_2_id','geo_level_3_id','land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status','count_floors_pre_eq', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

In [ ]:
train_values.info()

## Creating the Model

In [ ]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

from catboost import Pool, CatBoostClassifier, cv
import catboost
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

## CatBoost Classifier

In [ ]:
def Snippet_200(): 
    print()
    print(format('How to find optimal parameters for CatBoost using GridSearchCV for Classification','*^82'))    
    # load libraries
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from catboost import CatBoostClassifier

    # load the iris datasets
    X = train_values; y = train_labels
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    
    print('Data loaded')
    model = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols)
    
    parameters = {'depth':[2,4,6,8,10,12],
                  'iterations':[10,100,500,1000,5000],
                  'learning_rate':[0.02,0.05,0.06,0.07], 
                  'l2_leaf_reg':[3,5,7,9],
                  'border_count':[11,13,15,17]}
    randm = GridSearchCV(estimator=model, param_grid = parameters,cv = 2)
    print('Paramaters defined')
    randm.fit(X_train, y_train)

    # Results from Random Search
    print("\n========================================================")
    print(" Results from Random Search " )
    print("========================================================")    

    print("\n The best estimator across ALL searched params:\n",
          randm.best_estimator_)

    print("\n The best score across ALL searched params:\n",
          randm.best_score_)

    print("\n The best parameters across ALL searched params:\n",
           randm.best_params_)

    print("\n ========================================================")
Snippet_200()

In [ ]:
from sklearn.metrics import f1_score
print("Trying on test set")
X = train_values; y = train_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
#input best params here
model = CatBoostClassifier(eval_metric='TotalF1',random_seed=42, task_type="GPU",cat_features=cat_cols,
                           depth=5, iterations=500 ,learning_rate=0.01,l2_leaf_reg=9,
                           border_count=10)
model.fit(X_train, y_train)
preds=model.predict(X_test)
accuracy=f1_score(y_test,preds,average='micro')
print("Score on test set")
print("\n ========================================================")
print(accuracy)

In [ ]:
model.save_model(DATA_DIR+'/catbost.sav')

## Predicting on the test set

In [ ]:
test_values = pd.read_csv(DATA_DIR+'/test_values.csv', index_col='building_id')
test_values=test_values.drop(columns=['has_secondary_use_institution',
                           'has_secondary_use_school', 'has_secondary_use_industry',
                           'has_secondary_use_use_police', 'has_secondary_use_other',
                          'has_secondary_use_health_post', 'has_secondary_use_gov_office'])
                          

predictions = model.predict(test_values)
predictions

In [ ]:
submission_format = pd.read_csv(DATA_DIR+'/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission['damage_grade'] =my_submission['damage_grade'].astype(int)
my_submission.head()

In [ ]:
my_submission.to_csv('tsecvsubmission.csv')
#!head submission.csv

## Review Best features

In [ ]:
df_importances=pd.DataFrame(train_values.columns)
df_importances['importances']=pipe.feature_importances_
df_importances[df_importances['importances']>0]
#df_importances.to_csv(DATA_DIR+"/df_importances.csv")

In [ ]:
selected_features_update=df_importances[0][df_importances['importances']>5]

In [ ]:
selected_features_update.to_list()
len(selected_features_update)

## Reload model


In [ ]:
from_file = CatBoostClassifier()

from_file.load_model(DATA_DIR+'/catbost.sav')
